# MiniSOM para un problema de Clasificación (spoiler)

In [2]:
from minisom import MiniSom
import numpy as np
import pandas as pd

In [13]:
dataset = pd.read_csv("11_extra_comprar_alquilar.csv")
data = dataset.iloc[:,:-1].values
labels = dataset.iloc[:,-1:].values

Función de clasificación que clasifica una muestra en datos usando la etiqueta asignada a la neurona ganadora asociada. Una etiqueta está asociada a una neurona si la mayoría de las muestras mapeadas en esa neurona tienen etiqueta. La función asignará la etiqueta más común en el conjunto de datos en caso de que una muestra se asigne a una neurona para la que no se haya asignado ninguna clase.

In [14]:
def classify(som, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(X_train, y_train)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

In [19]:
def classify(som, data, labels):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(data, labels.ravel())
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

Ahora podemos 1) dividir los datos en train y conjunto de prueba, 2) entrenar el som, 3) imprimir el informe de clasificación que contiene todas las métricas para evaluar los resultados de la clasificación.

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(data, labels, stratify=labels)

som = MiniSom(7, 7, data.shape[1], sigma=3, learning_rate=0.5, random_seed=10)
som.pca_weights_init(X_train)
som.train_random(X_train, 500, verbose=False)

print(classification_report(y_test.ravel(), classify(som, X_test, y_test)))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93        34
           1       0.88      0.82      0.85        17

    accuracy                           0.90        51
   macro avg       0.89      0.88      0.89        51
weighted avg       0.90      0.90      0.90        51



1. División de los datos en conjuntos de entrenamiento y prueba

Se dividen los datos en conjuntos de entrenamiento y prueba usando la función train_test_split. El conjunto de entrenamiento se utiliza para entrenar el modelo, mientras que el conjunto de prueba se utiliza para evaluar su rendimiento. La opción stratify=labels asegura que la distribución de las etiquetas en los conjuntos de entrenamiento y prueba sea similar.

2. Inicialización y entrenamiento de la red SOM

Se inicializa una red SOM con una cuadrícula de 7x7 utilizando la función MiniSom. Los parámetros como sigma, learning_rate y random_seed son parámetros específicos de SOM.

Se inicializan los pesos de la red SOM utilizando el análisis de componentes principales (PCA) de los datos de entrenamiento (som.pca_weights_init(X_train)). En el contexto de las Redes de Kohonen o SOM, PCA se utiliza a menudo para inicializar los pesos de la red. Esto puede ayudar a acelerar el entrenamiento de la SOM y puede llevar a una representación más significativa de los datos.

En este caso, som.pca_weights_init(X_train) inicializa los pesos de la red SOM utilizando los primeros dos componentes principales de los datos de entrenamiento. Esto tiene el efecto de colocar los nodos o neuronas de la SOM a lo largo de las direcciones de máxima variabilidad en los datos.

Es importante tener en cuenta que aunque PCA puede ser una forma útil de inicializar los pesos de una SOM, no es la única opción. Los pesos también pueden ser inicializados aleatoriamente o usando otras técnicas. La elección de la inicialización de los pesos puede tener un impacto significativo en los resultados del entrenamiento de la SOM.



Luego, la red SOM se entrena con los datos de entrenamiento utilizando la función som.train_random.

3. Clasificación de los datos de prueba

La función classify se utiliza para clasificar los datos de prueba. Esta función asigna cada dato a la neurona ganadora en la red SOM. Si la neurona ganadora tiene una etiqueta asignada (basada en los datos de entrenamiento), se asigna esa etiqueta al dato. Si no, se asigna la etiqueta más común en los datos de entrenamiento.

La función som.labels_map(X_train, y_train) se utiliza para asignar etiquetas a las neuronas ganadoras basándose en los datos de entrenamiento.

4. Evaluación del modelo

Por último, se utiliza la función classification_report para generar un informe de las métricas de clasificación. Esto proporciona medidas útiles como precisión, recall y f1-score, que pueden usarse para evaluar la calidad del modelo de clasificación.